In [1]:
import digits
import tensorflow as tf
from time import time
import itertools as it
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
DATA_PATH = 'digits.csv'

LEARNING_RATE = 0.1
BATCH_SIZE = 128
EPOCHS = 30
MODEL_DIR = 'cnn-da/model_{}'

In [3]:
labels, pixels = digits.read(DATA_PATH)
pixels_sq = pixels.reshape(-1, 28, 28)

In [4]:
train_label, test_label, train_pixel, test_pixel = train_test_split(labels, pixels_sq, test_size=0.1, 
                                                                    stratify=labels, shuffle=True, random_state=0)
train_label, dev_label, train_pixel, dev_pixel = train_test_split(train_label, train_pixel, test_size=0.11111111, 
                                                                  stratify=train_label, shuffle=True, random_state=0)

In [5]:
train_label.shape, dev_label.shape, test_label.shape

((8000,), (1000,), (1000,))

In [6]:
train_pixel.shape, dev_pixel.shape, test_pixel.shape

((8000, 28, 28), (1000, 28, 28), (1000, 28, 28))

In [7]:
num_labels = len(set(labels))
num_labels

10

In [8]:
def model_fn(features, labels, mode):
    c1 = tf.layers.conv2d(features, 4, 3, activation=tf.nn.relu) # 26 x 26
    c2 = tf.layers.conv2d(c1, 4, 3, activation=tf.nn.relu) # 24 x 24
    m1 = tf.layers.max_pooling2d(c2, 2, 2) # 12 x 12
    c3 = tf.layers.conv2d(m1, 8, 3, activation=tf.nn.relu) # 10 x 10
    c4 = tf.layers.conv2d(c3, 8, 3, activation=tf.nn.relu) # 8 x 8
    c5 = tf.layers.conv2d(c4, 8, 3, activation=tf.nn.relu) # 6 x 6
    m2 = tf.layers.max_pooling2d(c5, 2, 2) # 3 x 3
    fc = tf.layers.flatten(m2)
    logits = tf.layers.dense(fc, num_labels)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        proba = tf.nn.softmax(logits, axis=-1)
        predictions = {
            'logits': logits,
            'probabilities': proba,
            'labels': tf.argmax(proba, axis=-1),
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.softmax_cross_entropy(labels, logits)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss)
    
    optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
    train_op = optimizer.minimize(loss, tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

In [9]:
import math

def augment_data(p, lbl, rotations):
    def rotate_processing(angle, p, lbl):
        if angle != 0:
            prot = tf.expand_dims(p, axis=-1)
            prot = tf.contrib.image.rotate(prot, angle / 180 * math.pi)
            prot = tf.squeeze(prot, axis=-1)
        else:
            prot = p
        return prot, lbl
    
    all_data = [rotate_processing(r, p, lbl) for r in rotations]
    all_pixels, all_labels = zip(*all_data)
    all_pixels = tf.concat(all_pixels, axis=0)
    all_labels = tf.concat(all_labels, axis=0)
    return all_pixels, all_labels

In [10]:
def td_input_fn(pixels, labels, batch_size, epochs, is_training, buffer=100):
    if is_training:
        pixels, labels = augment_data(pixels, labels, [-10, -5, 0, 5, 10])
    
    ds = tf.data.Dataset.from_tensor_slices((pixels, labels))
    
    def preprocessing(p, lbl):
        p3d = tf.expand_dims(p, axis=-1)
        p3d = tf.to_float(p3d)
        p3d = tf.image.per_image_standardization(p3d)
        lbl_1h = tf.one_hot(lbl, num_labels, dtype=tf.float32)
        return p3d, lbl_1h
    
    ds = ds.map(preprocessing, num_parallel_calls=8)
    
    if is_training:
        ds = ds.shuffle(buffer * batch_size)
        
    ds = ds.batch(batch_size)
    ds = ds.repeat(epochs)
    ds = ds.prefetch(buffer)
    
    return ds

In [11]:
def pred_input_fn(pixels, batch_size, buffer=100):
    ds = tf.data.Dataset.from_tensor_slices(pixels)
    
    def preprocessing(p):
        p3d = tf.expand_dims(p, axis=-1)
        p3d = tf.to_float(p3d)
        p3d = tf.image.per_image_standardization(p3d)
        return p3d
        
    ds = ds.map(preprocessing, num_parallel_calls=8)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer)
    
    return ds

In [12]:
model_dir = MODEL_DIR.format(int(time()))
config = tf.estimator.RunConfig(tf_random_seed=0,
                                save_summary_steps=50,
                                save_checkpoints_steps=250,
                                keep_checkpoint_max=None)

clf = tf.estimator.Estimator(model_fn, model_dir, config)

INFO:tensorflow:Using config: {'_model_dir': 'cnn-da/model_1540252757', '_tf_random_seed': 0, '_save_summary_steps': 50, '_save_checkpoints_steps': 250, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000027FFF7C62E8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
train_input_fn = lambda: td_input_fn(train_pixel, train_label, BATCH_SIZE, EPOCHS, True)
dev_input_fn = lambda: td_input_fn(dev_pixel, dev_label, BATCH_SIZE, 1, False)

train_spec = tf.estimator.TrainSpec(train_input_fn)
eval_spec = tf.estimator.EvalSpec(dev_input_fn, None, start_delay_secs=0.1, throttle_secs=0.1)

In [14]:
tf.estimator.train_and_evaluate(clf, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 250 or save_checkpoints_secs None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into cnn-da/model_1540252757\model.ckpt.
INFO:tensorflow:cnn-da/model_1540252757\model.ckpt-0 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:loss = 2.3976583, step = 0
INFO:tensorflow:global_step/sec: 64.8769
INFO:tensorflow:loss = 2.2908745, step = 100 (1.557 sec)
INFO:tensorflow:global_step/sec: 124.694
INFO:tensorflow:loss = 0.52643114, step = 200 (0.786 sec)
INFO:tensorflow:Saving checkpoints f

INFO:tensorflow:cnn-da/model_1540252757\model.ckpt-2000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:00:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-da/model_1540252757\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:00:41
INFO:tensorflow:Saving dict for global step 2000: global_step = 2000, loss = 0.15097567
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: cnn-da/model_1540252757\model.ckpt-2000
INFO:tensorflow:global_step/sec: 38.432
INFO:tensorflow:loss = 0.25628838, step = 2000 (2.602 sec)
INFO:tensorflow:global_step/sec: 124.658
INFO:tensorflow:loss = 0.055020317, step = 2100 (0.802 sec)
INFO:tensorflow:global_step/sec: 127.169
INFO:tensorflow:loss = 0.10490154, step = 2200 (0.786

INFO:tensorflow:Saving checkpoints for 4000 into cnn-da/model_1540252757\model.ckpt.
INFO:tensorflow:cnn-da/model_1540252757\model.ckpt-4000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:01:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-da/model_1540252757\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:01:11
INFO:tensorflow:Saving dict for global step 4000: global_step = 4000, loss = 0.1078677
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4000: cnn-da/model_1540252757\model.ckpt-4000
INFO:tensorflow:global_step/sec: 39.0331
INFO:tensorflow:loss = 0.05830109, step = 4000 (2.562 sec)
INFO:tensorflow:global_step/sec: 128.03
INFO:tensorflow:loss = 0.1036389, step = 4100 (0.781 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 120.779
INFO:tensorflow:loss = 0.030036949, step = 5900 (0.828 sec)
INFO:tensorflow:Saving checkpoints for 6000 into cnn-da/model_1540252757\model.ckpt.
INFO:tensorflow:cnn-da/model_1540252757\model.ckpt-6000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:01:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-da/model_1540252757\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:01:41
INFO:tensorflow:Saving dict for global step 6000: global_step = 6000, loss = 0.12035553
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6000: cnn-da/model_1540252757\model.ckpt-6000
INFO:tensorflow:global_step/sec: 38.7975
INFO:tensorflow:loss = 0.02853736, step = 6000 (2.577 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 36.3722
INFO:tensorflow:loss = 0.09976102, step = 7800 (2.749 sec)
INFO:tensorflow:global_step/sec: 130.643
INFO:tensorflow:loss = 0.040769625, step = 7900 (0.765 sec)
INFO:tensorflow:Saving checkpoints for 8000 into cnn-da/model_1540252757\model.ckpt.
INFO:tensorflow:cnn-da/model_1540252757\model.ckpt-8000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:02:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-da/model_1540252757\model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:02:11
INFO:tensorflow:Saving dict for global step 8000: global_step = 8000, loss = 0.117905274
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8000: cnn-da/model_1540252757\model.ckpt-8000
INFO:te

({'loss': 0.09483148, 'global_step': 9390}, [])

In [17]:
checkpoint_path = r'cnn-da/model_1540252757\model.ckpt-9390'

In [18]:
test_input_fn = lambda: pred_input_fn(test_pixel, BATCH_SIZE)

test_pred = clf.predict(test_input_fn, predict_keys=['labels'], checkpoint_path=checkpoint_path)
test_pred = [p['labels'] for p in test_pred]
test_pred = np.asarray(test_pred)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-da/model_1540252757\model.ckpt-9390
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [19]:
eval_input_fn = lambda: td_input_fn(test_pixel, test_label, BATCH_SIZE, 1, False)
clf.evaluate(eval_input_fn, checkpoint_path=checkpoint_path)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-23-00:05:00
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cnn-da/model_1540252757\model.ckpt-9390
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-23-00:05:00
INFO:tensorflow:Saving dict for global step 9390: global_step = 9390, loss = 0.07915214
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9390: cnn-da/model_1540252757\model.ckpt-9390


{'loss': 0.07915214, 'global_step': 9390}

In [20]:
print(classification_report(test_label, test_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99        99
           1       0.99      0.97      0.98       110
           2       1.00      0.99      1.00       104
           3       0.98      1.00      0.99       101
           4       0.96      0.95      0.95        97
           5       1.00      0.98      0.99        91
           6       0.99      0.96      0.97       100
           7       0.97      0.96      0.97       104
           8       0.96      0.98      0.97        95
           9       0.92      0.97      0.95        99

   micro avg       0.98      0.98      0.98      1000
   macro avg       0.98      0.98      0.98      1000
weighted avg       0.98      0.98      0.98      1000



In [21]:
cm = confusion_matrix(test_label, test_pred)
cm

array([[ 99,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 107,   0,   0,   1,   0,   0,   0,   2,   0],
       [  0,   0, 103,   0,   0,   0,   0,   1,   0,   0],
       [  0,   0,   0, 101,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,  92,   0,   0,   1,   0,   4],
       [  0,   0,   0,   0,   0,  89,   1,   0,   0,   1],
       [  1,   0,   0,   0,   1,   0,  96,   0,   2,   0],
       [  0,   1,   0,   0,   1,   0,   0, 100,   0,   2],
       [  0,   0,   0,   1,   0,   0,   0,   0,  93,   1],
       [  0,   0,   0,   1,   1,   0,   0,   1,   0,  96]], dtype=int64)